# Imports

In [1111]:
#from basketball_reference web_scraper 
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import OutputType

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Scraping Player Stats for a Total Season

In [1112]:
def get_player_total_season_stats(year):
    """
    Retrieves player season total stats for a given NBA season and saves them to a CSV file.
    
    This function uses an API wrapper to fetch player season total statistics for the specified NBA season, saves the data to a CSV file, drops the 'slug' column from the dataset, and then saves the updated dataset back to the CSV file.
    
    Parameters:
    -----------
    year : int
        The ending year of the NBA season for which to retrieve player statistics. For example, to get statistics for the 2022-2023 season, pass 2023 as the year.
    
    Returns:
    --------
    None
        It saves the player season total statistics to a CSV file.

    Example:
    --------
    To retrieve and save player statistics for the 2022-2023 NBA season:
    >>> get_player_total_season_stats(2023)
    """
    #from the api wrapper
    client.players_season_totals(
        season_end_year= year, 
        output_type=OutputType.CSV, 
        output_file_path=f"./total_stats/{year-1}_{year}_player_season_totals.csv"
    )
    #dropping slug column
    df = pd.read_csv(f'./total_stats/{year-1}_{year}_player_season_totals.csv')
    df.drop(columns = 'slug', inplace = True)
    return df.to_csv(f"./total_stats/{year-1}_{year}_player_season_totals.csv", index = False)

# 2022/2023 Season Total Stats EDA

Not running the below cell again as I have already scraped the data.

In [1113]:
#eda for player totals
df_total = pd.read_csv('./total_stats/2022_2023_player_season_totals.csv')
df_total.isnull().sum()

name                                 0
positions                            0
age                                  0
team                                 0
games_played                         0
games_started                        0
minutes_played                       0
made_field_goals                     0
attempted_field_goals                0
made_three_point_field_goals         0
attempted_three_point_field_goals    0
made_free_throws                     0
attempted_free_throws                0
offensive_rebounds                   0
defensive_rebounds                   0
assists                              0
steals                               0
blocks                               0
turnovers                            0
personal_fouls                       0
points                               0
dtype: int64

No null values.

In [1114]:
df_total

,name,positions,age,team,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
0,Precious Achiuwa,CENTER,23,TORONTO RAPTORS,55,12,1140,196,404,29,108,87,124,100,228,50,31,30,59,102,508
1,Steven Adams,CENTER,29,MEMPHIS GRIZZLIES,42,42,1133,157,263,0,1,47,129,214,271,97,36,46,79,98,361
2,Bam Adebayo,CENTER,25,MIAMI HEAT,75,75,2598,602,1114,1,12,324,402,184,504,240,88,61,187,208,1529
3,Ochai Agbaji,SHOOTING GUARD,22,UTAH JAZZ,59,22,1209,165,386,81,228,56,69,43,78,67,16,15,41,99,467
4,Santi Aldama,POWER FORWARD,22,MEMPHIS GRIZZLIES,77,20,1682,247,525,94,266,108,144,85,286,97,45,48,60,143,696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,Thaddeus Young,POWER FORWARD,34,TORONTO RAPTORS,54,9,795,108,198,6,34,18,26,71,95,75,54,5,42,88,240
605,Trae Young,POINT GUARD,24,ATLANTA HAWKS,73,73,2541,597,1390,154,460,566,639,56,161,741,80,9,300,104,1914
606,Omer Yurtseven,CENTER,24,MIAMI HEAT,9,0,83,16,27,3,7,5,6,8,15,2,2,2,4,16,40
607,Cody Zeller,CENTER,30,MIAMI HEAT,15,2,217,37,59,0,2,24,35,25,39,10,3,4,14,33,98


In [1115]:
df_total.describe()

,age,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
count,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000
mean,25.893268,42.518883,20.197044,976.743842,169.556650,356.681445,49.850575,138.201970,74.298851,95.042693,42.149425,133.315271,102.264368,29.446634,18.798030,54.252874,80.704433,463.262726
std,4.312631,25.081185,25.918279,807.544477,172.127488,356.427911,57.390304,152.025741,100.816793,123.998786,49.555581,132.381970,122.442202,27.244221,25.341943,56.206109,65.177501,479.765348
min,19.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,21.000000,0.000000,240.000000,32.000000,70.000000,4.000000,14.000000,10.000000,13.000000,8.000000,30.000000,17.000000,7.000000,4.000000,10.000000,23.000000,85.000000
50%,25.000000,43.000000,5.000000,786.000000,108.000000,230.000000,27.000000,86.000000,34.000000,45.000000,26.000000,92.000000,56.000000,22.000000,11.000000,36.000000,67.000000,288.000000
75%,29.000000,65.000000,38.000000,1670.000000,257.000000,530.000000,74.000000,216.000000,96.000000,129.000000,57.000000,197.000000,137.000000,45.000000,24.000000,81.000000,125.000000,694.000000
max,42.000000,82.000000,82.000000,2842.000000,728.000000,1559.000000,301.000000,731.000000,669.000000,772.000000,274.000000,744.000000,741.000000,128.000000,193.000000,300.000000,279.000000,2225.000000


Nothing out of the ordinary.

In [1116]:
df_total.shape

(609, 21)

In [1117]:
df_total['name'].nunique()

539

It seems we have duplicate players if they got traded midseason. We need to combine these statistics.

In [1118]:
df_total['name'].value_counts()

name
Dorian Finney-Smith    2
Thomas Bryant          2
Spencer Dinwiddie      2
Moses Brown            2
Eugene Omoruyi         2
                      ..
James Harden           1
Tim Hardaway Jr.       1
Jordan Hall            1
Tyrese Haliburton      1
Ivica Zubac            1
Name: count, Length: 539, dtype: int64

In [1119]:
def combine_traded_player_total_stats(df):
    """
    Combines statistics for NBA players who were traded during the season for the player season total stats dataframe.
    
    This function identifies players who were traded during the season (i.e., appear more than once in the dataset with different teams), combines their statistics into a single row per player, and updates the DataFrame accordingly.
    
    Parameters:
    -----------
    df: pandas.DataFrame
        A DataFrame containing player total stats for a season.
        
    Returns:
    --------
    df: pandas.DataFrame
        The updated DataFrame with combined statistics for traded players.
        
    Example:
    --------
    To combine the stats for players who were traded during the 2022-2023 season:
    >>> df = pd.read_csv('2022_2023_player_season_totals.csv')
    >>> df_combined = combine_traded_player_total_stats(df)
    """
    player = df['name'].value_counts()
    multi_player = player[player > 1].index
    #create a new dataframe that just has the duplicate payers in it
    multi_df = df[df['name'].isin(multi_player)]
    df = df[df['name'].apply(lambda x: x not in multi_player)]

    #combine the stats from the duplicate players
    combo_stats = multi_df.groupby('name')[['games_played', 'games_started', 'minutes_played', 'made_field_goals', 'attempted_field_goals', 'made_three_point_field_goals', 'attempted_three_point_field_goals', 'made_free_throws', 'attempted_free_throws', 'offensive_rebounds', 'defensive_rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points']].sum().reset_index()
    team_names = multi_df.groupby('name')['team'].apply(list).reset_index()
    team_names['team_1'] = team_names['team'].apply(lambda x: x[0])
    team_names['team_2'] = team_names['team'].apply(lambda x: x[1] if len(x) > 1 else None)
    team_names = team_names.drop(columns='team')

    # Merge the combined stats with team names
    combo_df = pd.merge(combo_stats, team_names, on='name')
    #only take the first position since it should be the same
    combo_positions = multi_df.groupby('name')['positions'].first().reset_index()
    #combine the dataframes
    combo_df = pd.merge(combo_df, combo_positions, on='name')
    #only take the first age since we dont want to sum them
    age = multi_df.groupby('name')['age'].first().reset_index()
    #combine dataframe
    combo_df = pd.merge(combo_df, age, on='name')
    #concat the dataframe with the combined stats with the old dataframe.
    df = pd.concat([df, combo_df], axis = 0).reset_index(drop = True)
    return df

In [1120]:
df_total = combine_traded_player_total_stats(df_total)

In [1121]:
def clean_total_stats(df):
    """
    Cleans the 'team' and 'positions' columns in the player total stats DataFrame.

    This function converts the values in the 'team' and 'positions' columns to title case to ensure consistent formatting.

    Parameters:
    -----------
    df: pandas.DataFrame
        A DataFrame containing player total stats for a season.
    Returns:
    --------
    df: pandas.DataFrame
        The updated DataFrame with the 'team' and 'positions' columns converted to title case.

    Example:
    --------
    To clean the 'team' and 'positions' columns in the DataFrame:
    >>> df = pd.read_csv('2022_2023_player_season_totals.csv')
    >>> df_cleaned = clean_total_stats(df)
    """
    #make the positions and team columns title case
    df['team'] = df['team'].str.title()
    df['team_1'] = df['team_1'].str.title()
    df['team_2'] = df['team_2'].str.title()
    df['positions'] = df['positions'].str.title()
    
    #Deal with 76Ers and correcting it to 76ers
    sixers = {
        'Philadelphia 76Ers':'Philadelphia 76ers'
    }
    df['team'] = df['team'].replace(sixers)
    df['team_1'] = df['team_1'].replace(sixers)
    df['team_2'] = df['team_2'].replace(sixers)
    return df

In [1122]:
#clean the dataframe
df_total = clean_total_stats(df_total)

# Scraping Player Advanced Stats

In [1123]:
def get_player_advanced_season_stats(year):
    """
    Retrieves and processes advanced player stats for a given season.

    This function uses an API to retrieve advanced player stats for the specified season, saves the data to a CSV file, and then processes the data by removing unnecessary columns.

    Parameters:
    -----------
    year : int
        The ending year of the season for which to retrieve the advanced player statistics.

    Returns:
    --------
    None
        It saves the player season total statistics to a CSV file.

    Example:
    --------
    To retrieve and save advanced player statistics for the 2022-2023 season:
    >>> get_player_advanced_season_stats(2023)
    """
    client.players_advanced_season_totals(
        season_end_year=year,
        output_type=OutputType.CSV,
        output_file_path=f"./advanced_season_stat_total/{year-1}_{year}_advanced_player_season_totals.csv"
)
#dropping duplicate and not needed columns 
    df = pd.read_csv(f'./advanced_season_stat_total/{year-1}_{year}_advanced_player_season_totals.csv')
    df.drop(columns = ['slug', 'positions', 'age', 'team', 'minutes_played', 'is_combined_totals'] , inplace = True)
    return df.to_csv(f"./advanced_season_stat_total/{year-1}_{year}_advanced_player_season_totals.csv", index = False)

Not running the below cell again as I have already scraped the data.

# Player Advanced Stats EDA

In [1124]:
df_advanced = pd.read_csv('./advanced_season_stat_total/2022_2023_advanced_player_season_totals.csv')
df_advanced.isnull().sum()

name                             0
games_played                     0
player_efficiency_rating         0
true_shooting_percentage         0
three_point_attempt_rate         0
free_throw_attempt_rate          0
offensive_rebound_percentage     0
defensive_rebound_percentage     0
total_rebound_percentage         0
assist_percentage                0
steal_percentage                 0
block_percentage                 0
turnover_percentage              0
usage_percentage                 0
offensive_win_shares             0
defensive_win_shares             0
win_shares                       0
win_shares_per_48_minutes        0
offensive_box_plus_minus         0
defensive_box_plus_minus         0
box_plus_minus                   0
value_over_replacement_player    0
dtype: int64

No null values present.

In [1125]:
df_advanced.describe()

,games_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player
count,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000
mean,42.518883,13.263218,0.559920,0.405148,0.245982,5.172578,14.996880,10.082102,13.333333,1.543350,1.805747,12.622003,18.382759,1.074713,0.990312,2.065846,0.081337,-1.374548,-0.075041,-1.450082,0.487521
std,25.081185,6.237213,0.114687,0.222474,0.181543,4.280231,6.837965,4.806279,8.735297,1.316029,2.359224,7.884842,5.835549,1.688998,0.965262,2.460253,0.088172,3.607156,2.131476,4.774525,1.141405
min,1.000000,-20.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.900000,0.000000,-1.600000,-0.517000,-22.500000,-10.400000,-26.500000,-1.300000
25%,21.000000,10.000000,0.522000,0.268000,0.138000,2.100000,10.700000,6.700000,7.300000,1.000000,0.700000,9.200000,14.300000,0.000000,0.200000,0.200000,0.045000,-3.000000,-0.900000,-3.300000,-0.100000
50%,43.000000,13.000000,0.566000,0.414000,0.227000,3.800000,13.400000,8.900000,11.100000,1.400000,1.200000,11.800000,17.700000,0.500000,0.700000,1.200000,0.086000,-1.400000,-0.100000,-1.300000,0.100000
75%,65.000000,16.300000,0.610000,0.553000,0.323000,7.100000,18.700000,12.600000,17.700000,1.800000,2.300000,15.100000,21.300000,1.500000,1.600000,3.100000,0.129000,0.500000,0.800000,0.600000,0.700000
max,82.000000,65.600000,1.064000,1.000000,2.000000,28.800000,55.400000,29.600000,47.600000,24.200000,44.100000,100.000000,52.500000,11.200000,4.800000,14.900000,0.626000,17.000000,32.700000,48.600000,8.800000


Nothing out of the ordinary.

In [1126]:
df_advanced.shape

(609, 22)

In [1127]:
df_advanced['name'].nunique()

539

Same issue as before. We have duplicate rows for players that were traded midseason.

In [1128]:
def combine_traded_player_advanced_stats(df):
    """
    Combines advanced stats for players who were traded during a season.

    This function takes a DataFrame containing advanced player stats and combines the stats for players who were traded during the season. It then calculates the weighted average of the combined stats based on the number of games played on each team.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing advanced player statistics.

    Returns:
    --------
    pandas.DataFrame
        DataFrame with combined/weighted averaged advanced player statistics for players who were traded during the season.

    Example:
    --------
    To combine the stats for players who were traded during the 2022-2023 season:
    >>> df = pd.read_csv('2022_2023_advanced_player_season_totals.csv')
    >>> df_combined = combine_traded_player_total_stats(df)
    """
    player = df['name'].value_counts()
    multi_player = player[player > 1].index
    #create a new dataframe that just has the duplicate payers in it
    multi_df = df[df['name'].isin(multi_player)]
    df = df[df['name'].apply(lambda x: x not in multi_player)]

    #combine the stats from the duplicate players
    combo_stats = multi_df.groupby('name')[['player_efficiency_rating', 'true_shooting_percentage', 'three_point_attempt_rate', 'free_throw_attempt_rate', 'offensive_rebound_percentage', 'defensive_rebound_percentage', 'total_rebound_percentage', 'assist_percentage', 'steal_percentage', 'block_percentage', 'turnover_percentage', 'usage_percentage', 'offensive_win_shares', 'defensive_win_shares', 'win_shares', 'win_shares_per_48_minutes', 'offensive_box_plus_minus', 'defensive_box_plus_minus', 'box_plus_minus', 'value_over_replacement_player']].apply(lambda x: x.multiply(multi_df.loc[x.index, 'games_played'], axis=0)).groupby('name').sum()
    #combine total games for each player
    total_games = multi_df.groupby('name')['games_played'].sum()
    
    #calculate the averages
    combo_stats = combo_stats.divide(total_games, axis=0).reset_index()
    #concat the dataframe with the combined stats with the old dataframe.
    df = pd.concat([df, combo_stats], axis = 0).reset_index(drop = True)
    df.drop(columns = 'games_played', inplace = True)
    return df

In [1129]:
df_advanced = combine_traded_player_advanced_stats(df_advanced)

In [1130]:
#combine the total and advanced stats dataframes
df = pd.merge(df_advanced, df_total, on = 'name')

In [1131]:
#confirming the correct shape
df.shape

(539, 43)

# Scrape Player Per Game Stats Per Season

In [1132]:
def get_per_game_stat(year):
    """
    Scrapes per-game basketball stats from Basketball Reference for a specific year and saves to as a pandas DataFrame.

    This function scrapes per-game stats of NBA players from Basketball Reference for a specified year. It retrieves stats such as player name, minutes per game (mpg), field goals made per game (fgm_per_g),
    field goals attempted per game (fga_per_g), field goal percentage (fg_pct), three-point field goals made per game (fg3m_per_g), three-point field goals attempted per game (fg3a_per_g), three-point field goal percentage (fg3_pct),
    two-point field goals made per game (fg2m_per_g), two-point field goals attempted per game (fg2a_per_g), two-point field goal percentage (fg2_pct), effective field goal percentage (efg_pct), free throws made per game (ftm_per_g), 
    free throws attempted per game (fta_per_g), free throw percentage (ft_pct), offensive rebounds per game (orb_per_g), defensive rebounds per game (drb_per_g), total rebounds per game (trb_per_g), assists per game (ast_per_g),
    steals per game (stl_per_g), blocks per game (blk_per_g), turnovers per game (tov_per_g), personal fouls per game (pf_per_g), and points per game (ppg).

    Parameters:
    -----------
    year : int
        The ending year of the season for which to retrieve the per-game stats.

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing per-game stats for NBA players in the specified year.

    Example:
    --------
    To scrape per-game stats for the year 2022-2023:
    >>> per_game_stats_2022 = get_per_game_stat(2023)
    """
    req = requests.get(f'https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html')
    soup = BeautifulSoup(req.content, 'html.parser')
    per_game_stat = []
    
    #the stats that I want in my dataframe
    stats = {
        'player': 'name',
        'mp_per_g' : 'mpg',
        'fg_per_g' : 'fgm_per_g',
        'fga_per_g' : 'fga_per_g',
        'fg_pct' : 'fg_pct',
        'fg3_per_g' : 'fg3m_per_g',
        'fg3a_per_g' : 'fg3a_per_g',
        'fg3_pct' : 'fg3_pct',
        'fg2_per_g' : 'fg2m_per_g',
        'fg2a_per_g' : 'fg2a_per_g',
        'fg2_pct' : 'fg2_pct',
        'efg_pct' : 'efg_pct',
        'ft_per_g' : 'ftm_per_g',
        'fta_per_g' : 'fta_per_g',
        'ft_pct' : 'ft_pct',
        'orb_per_g': 'orb_per_g',
        'drb_per_g' : 'drb_per_g',
        'trb_per_g' : 'trb_per_g',
        'ast_per_g' : 'ast_per_g',
        'stl_per_g' : 'stl_per_g',
        'blk_per_g' : 'blk_per_g',
        'tov_per_g' : 'tov_per_g',
        'pf_per_g' : 'pf_per_g',
        'pts_per_g' : 'ppg',
        #added team so that I can deal with traded players data
        'team_id': 'team'
    }
        
    #finding each player and their specific stat
    for stat in soup.find('tbody').find_all('tr'):
        per_game_stat_dict = {}
        player = stat.find('td', {'data-stat':'player'})
        #need to add if statement so it only recognizes player names that are filled out
        if player:
            player_found = player.find('a')
            if player_found:
                per_game_stat_dict['name'] = player_found.getText()
        #need to add a for loop to loop through the stats and find each key in the html dictionary
        for key, value in stats.items():
            if key != 'player':
                stat_value = stat.find('td', {'data-stat': key})
                if stat_value:
                    per_game_stat_dict[value] = stat_value.getText()  
        per_game_stat.append(per_game_stat_dict)
    #create a dataframe of our stats and drop any null values
    df_per_game = pd.DataFrame(per_game_stat)
    df_per_game.reset_index(drop=True, inplace=True)
    df_per_game.to_csv(f'./per_game_stat/per_game_stat_{year-1}_{year}.csv', index = False)
    return df_per_game

# EDA on Per Game Stats

Not running the below cell again as I have already scraped the data.

In [1133]:
df_per = pd.read_csv('./per_game_stat/per_game_stat_2022_2023.csv')

In [1134]:
def combine_traded_player_per_game_stats(df):
    """
    Combines per-game stats of players who were traded during the season.

    This function takes a DataFrame containing per-game stats and combines the stats of players who were traded during the season. It identifies players with multiple entries (indicating they were traded) 
    and combines their stats into a single entry.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing per-game stats of NBA players.

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing per-game stats with combined stats for traded players.

    Example:
    --------
    To combine the per-game stats for players who were traded during the 2022-2023 season:
    >>> df = pd.read_csv('per_game_stat_2022_2023.csv')
    >>> df_combined = combine_traded_player_per_game_stats(df)
    """
    #Identify players that show up multiple times
    player = df['name'].value_counts()
    multi_player = player[player > 1].index
    #create a new dataframe that just has the duplicate payers in it
    multi_df = df[df['name'].isin(multi_player)]
    df = df[df['name'].apply(lambda x: x not in multi_player)]
    grouped = multi_df.groupby('name')
    
    total_list = []
    #If the team is "total" then grab those values
    for player, stats in grouped:
        if 'TOT' in stats['team'].values:
            total = stats[stats['team'] == 'TOT']
            total_list.append(total)
            
    total = pd.concat(total_list, axis = 0)
    
    df = pd.concat([total, df], axis = 0).reset_index(drop = True)
    df.drop(columns = 'team', inplace = True)
    return df

In [1135]:
df_per = combine_traded_player_per_game_stats(df_per)

# Scrape NBA Team Data

In [1136]:
def scrape_team_data(year):
    #Scrape Basktball Reference
    req = requests.get(f'https://www.basketball-reference.com/leagues/NBA_{year}.html')
    soup = BeautifulSoup(req.content, 'html.parser')
    team_stat = []
    
    #Specify the stats we are interested in
    stats = {
        'wins': 'wins',
        'losses': 'losses',
        'win_loss_pct': 'win_loss_pct',
        'pts_per_g': 'team_pts_per_g',
        'opp_pts_per_g': 'opp_pts_per_g',
        'srs': 'team_simple_rating_system'
    }
    
    #Loop through both conferences
    for conference in ['E', 'W']:
        conference_table = soup.find('table', {'id': f'confs_standings_{conference}'})
        # Looping through the team names
        for stat in conference_table.find('tbody').find_all('tr'):
            team_stat_dict = {}
            team_name = stat.find('a')
            team_stat_dict['team'] = team_name.text
            #Loop through the stats for each team
            for key, value in stats.items():
                stat_value = stat.find('td', {'data-stat': key})
                team_stat_dict[value] = stat_value.getText()
            #Append back to our list
            team_stat.append(team_stat_dict)
    #Create dataframe
    df_team = pd.DataFrame(team_stat)
    df_team.reset_index(drop=True, inplace=True)
    df_team.to_csv(f'./team_stats/team_stats_{year-1}_{year}.csv', index=False)
    return df_team

In [1137]:
df_team = scrape_team_data(2023)
df_team

,team,wins,losses,win_loss_pct,team_pts_per_g,opp_pts_per_g,team_simple_rating_system
0,Milwaukee Bucks,58,24,.707,116.9,113.3,3.61
1,Boston Celtics,57,25,.695,117.9,111.4,6.38
2,Philadelphia 76ers,54,28,.659,115.2,110.9,4.37
3,Cleveland Cavaliers,51,31,.622,112.3,106.9,5.23
4,New York Knicks,47,35,.573,116.0,113.1,2.99
5,Brooklyn Nets,45,37,.549,113.4,112.5,1.03
6,Miami Heat,44,38,.537,109.5,109.8,-0.13
7,Atlanta Hawks,41,41,.500,118.4,118.1,0.32
8,Toronto Raptors,41,41,.500,112.9,111.4,1.59
9,Chicago Bulls,40,42,.488,113.1,111.8,1.37


# Combine to Make One DataFrame

In [1138]:
#combine the total and per year stats dataframes
df = pd.merge(df_per, df, on = 'name')

In [1139]:
#Need to one hot encode positions
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 66 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               539 non-null    object 
 1   mpg                                539 non-null    float64
 2   fgm_per_g                          539 non-null    float64
 3   fga_per_g                          539 non-null    float64
 4   fg_pct                             537 non-null    float64
 5   fg3m_per_g                         539 non-null    float64
 6   fg3a_per_g                         539 non-null    float64
 7   fg3_pct                            523 non-null    float64
 8   fg2m_per_g                         539 non-null    float64
 9   fg2a_per_g                         539 non-null    float64
 10  fg2_pct                            534 non-null    float64
 11  efg_pct                            537 non-null    float64

In [1140]:
df = pd.get_dummies(df, columns=['positions'], dtype = int)
df

,name,mpg,fgm_per_g,fga_per_g,fg_pct,fg3m_per_g,fg3a_per_g,fg3_pct,fg2m_per_g,fg2a_per_g,fg2_pct,efg_pct,ftm_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,ppg,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,age,team,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points,team_1,team_2,positions_Center,positions_Point Guard,positions_Power Forward,positions_Shooting Guard,positions_Small Forward
0,A.J. Lawson,7.2,1.5,2.9,0.500,0.7,1.7,0.400,0.8,1.3,0.632,0.614,0.1,0.5,0.250,0.4,1.0,1.4,0.1,0.1,0.0,0.2,0.7,3.7,14.893333,0.608000,0.542267,0.173600,6.066667,18.126667,12.126667,2.800000,0.840000,0.000000,5.693333,21.120000,0.093333,0.093333,0.093333,0.078333,-1.553333,-2.393333,-3.946667,-0.093333,22,NaN,15,0,108,22,44,10,25,2,8,6,15,2,2,0,3,11,56,Minnesota Timberwolves,Dallas Mavericks,0,0,0,1,0
1,Bones Hyland,19.4,4.1,10.2,0.399,2.1,5.6,0.371,2.0,4.6,0.434,0.502,1.5,1.8,0.843,0.4,2.1,2.4,3.1,0.7,0.3,1.5,1.7,11.8,14.975000,0.534500,0.551250,0.178250,2.125000,11.875000,7.100000,23.400000,1.775000,1.225000,11.700000,28.000000,0.125000,0.675000,0.800000,0.061750,0.400000,-1.225000,-0.825000,0.250000,22,NaN,56,1,1085,228,571,117,315,86,102,20,115,172,39,15,82,96,659,Denver Nuggets,Los Angeles Clippers,0,1,0,0,0
2,Bruno Fernando,10.4,1.5,2.9,0.527,0.0,0.1,0.000,1.5,2.8,0.546,0.527,0.9,1.3,0.700,1.4,2.1,3.5,0.8,0.2,0.9,0.6,1.9,3.9,16.920513,0.573923,0.041154,0.440795,14.607692,22.948718,18.728205,9.979487,0.635897,7.869231,15.858974,17.602564,0.397436,0.317949,0.656410,0.105872,-2.758974,0.382051,-2.376923,0.000000,24,NaN,39,4,405,59,112,0,4,35,50,54,83,31,6,34,25,74,153,Houston Rockets,Atlanta Hawks,1,0,0,0,0
3,Cam Reddish,24.8,3.5,7.8,0.446,1.1,3.6,0.313,2.4,4.2,0.560,0.518,1.6,1.9,0.853,0.5,1.8,2.2,1.4,1.0,0.4,1.2,1.8,9.7,10.600000,0.560500,0.456000,0.241000,2.000000,7.800000,4.900000,7.950000,1.900000,1.350000,11.600000,17.000000,0.150000,0.350000,0.500000,0.048000,-2.600000,-0.700000,-3.300000,-0.150000,23,NaN,40,20,990,139,312,45,144,64,75,18,70,57,39,14,47,72,387,New York Knicks,Portland Trail Blazers,0,0,0,0,1
4,Cameron Johnson,28.5,5.3,11.3,0.470,2.5,6.1,0.404,2.9,5.2,0.548,0.579,2.4,2.9,0.842,0.8,3.6,4.4,1.9,1.2,0.3,0.9,2.0,15.5,17.102381,0.617690,0.539857,0.249333,3.202381,13.935714,8.638095,9.554762,1.997619,1.061905,6.469048,20.492857,1.078571,0.738095,1.816667,0.143143,2.128571,0.600000,2.728571,0.719048,26,NaN,42,41,1199,223,474,103,255,101,120,34,151,78,50,14,37,83,650,Phoenix Suns,Brooklyn Nets,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Thaddeus Young,14.7,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,14.100000,0.573000,0.172000,0.131000,9.400000,14.600000,11.800000,12.900000,3.400000,0.600000,16.700000,13.500000,0.700000,1.100000,1.800000,0.109000,-1.800000,1.900000,0.100000,0.400000,34,Toronto Raptors,54,9,795,108,198,6,34,18,26,71,95,75,54,5,42,88,240,NaN,NaN,0,0,1,0,0
535,Trae Young,34.8,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,22.000000,0.573000,0.331000,0.460000,2.400000,7.000000,4.700000,42.50

In [1141]:
#merge the players who werent traded with the df_team
df_merge = pd.merge(df.dropna(subset=['team']), df_team, on='team', how = 'outer')

In [1142]:
def combine_traded_player_team_data(df, df_team, df_merge):
    df_traded = df[df['team'].isna()]
    # Merge player_df with team_stats_df for both team_1 and team_2
    df_traded = pd.merge(df_traded, df_team, left_on='team_1', right_on='team')
    df_traded = pd.merge(df_traded, df_team, left_on='team_2', right_on='team', suffixes=('', '_team_2'))

    columns = ['wins', 'wins_team_2', 'losses', 'losses_team_2', 'win_loss_pct', 'win_loss_pct_team_2', 'team_pts_per_g', 'team_pts_per_g_team_2', 'opp_pts_per_g', 'opp_pts_per_g_team_2', 'team_simple_rating_system', 'team_simple_rating_system_team_2']
    df_traded[columns] = df_traded[columns].astype(float)

    # Calculate the average stats
    df_traded['wins'] = (df_traded['wins'] + df_traded['wins_team_2']) / 2
    df_traded['losses'] = (df_traded['losses'] + df_traded['losses_team_2']) / 2
    df_traded['win_loss_pct'] = (df_traded['win_loss_pct'] + df_traded['win_loss_pct_team_2']) / 2
    df_traded['team_pts_per_g'] = (df_traded['team_pts_per_g'] + df_traded['team_pts_per_g_team_2']) / 2
    df_traded['opp_pts_per_g'] = (df_traded['opp_pts_per_g'] + df_traded['opp_pts_per_g_team_2']) / 2
    df_traded['team_simple_rating_system'] = (df_traded['team_simple_rating_system'] + df_traded['team_simple_rating_system_team_2']) / 2

    df_traded['team'] = df_traded.apply(lambda row: ', '.join([str(row['team_1']), str(row['team_2'])]), axis=1)
    df_traded.drop(columns = ['team_1', 'team_2', 'team_y', 'wins_team_2', 'losses_team_2', 'win_loss_pct_team_2', 'team_pts_per_g_team_2', 'opp_pts_per_g_team_2', 'team_simple_rating_system_team_2'], inplace = True)

    #concat the df merge with the players who were traded
    df_merge = pd.concat([df_merge, df_traded])
    df_merge.drop(columns = ['team_x', 'team_1', 'team_2'], inplace = True)
    return df_merge

In [ ]:
df_traded

,name,mpg,fgm_per_g,fga_per_g,fg_pct,fg3m_per_g,fg3a_per_g,fg3_pct,fg2m_per_g,fg2a_per_g,fg2_pct,efg_pct,ftm_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,ppg,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,age,team_x,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points,team_1,team_2,positions_Center,positions_Point Guard,positions_Power Forward,positions_Shooting Guard,positions_Small Forward,team_y,wins,losses,win_loss_pct,team_pts_per_g,opp_pts_per_g,team_simple_rating_system,team,wins_team_2,losses_team_2,win_loss_pct_team_2,team_pts_per_g_team_2,opp_pts_per_g_team_2,team_simple_rating_system_team_2
0,A.J. Lawson,7.2,1.5,2.9,0.500,0.7,1.7,0.400,0.8,1.3,0.632,0.614,0.1,0.5,0.250,0.4,1.0,1.4,0.1,0.1,0.0,0.2,0.7,3.7,14.893333,0.608000,0.542267,0.173600,6.066667,18.126667,12.126667,2.800000,0.840000,0.000000,5.693333,21.120000,0.093333,0.093333,0.093333,0.078333,-1.553333,-2.393333,-3.946667,-0.093333,22,NaN,15,0,108,22,44,10,25,2,8,6,15,2,2,0,3,11,56,Minnesota Timberwolves,Dallas Mavericks,0,0,0,1,0,Minnesota Timberwolves,40.0,42.0,0.4875,115.00,114.95,-0.180,"Minnesota Timberwolves, Dallas Mavericks",38.0,44.0,0.463,114.2,114.1,-0.14
1,Justin Holiday,15.3,1.7,4.4,0.377,1.0,3.1,0.322,0.7,1.3,0.508,0.490,0.1,0.2,0.625,0.1,1.1,1.2,0.9,0.4,0.4,0.5,1.5,4.5,6.608696,0.494217,0.701087,0.039522,0.773913,8.008696,4.352174,7.560870,1.304348,2.334783,9.582609,14.078261,-0.178261,0.239130,0.100000,0.007783,-3.686957,-0.169565,-3.856522,-0.200000,33,NaN,46,2,706,77,204,46,143,5,8,5,50,41,19,19,22,68,205,Atlanta Hawks,Dallas Mavericks,0,0,0,0,1,Atlanta Hawks,39.5,42.5,0.4815,116.30,116.10,0.090,"Atlanta Hawks, Dallas Mavericks",38.0,44.0,0.463,114.2,114.1,-0.14
2,Kyrie Irving,37.4,9.9,20.1,0.494,3.1,8.3,0.379,6.8,11.8,0.574,0.572,4.1,4.6,0.905,1.0,4.1,5.1,5.5,1.1,0.8,2.1,2.8,27.1,22.400000,0.614000,0.411667,0.227667,3.066667,12.266667,7.733333,25.333333,1.466667,1.733333,8.800000,28.933333,2.766667,1.133333,3.900000,0.159333,4.833333,-0.666667,4.166667,1.900000,30,NaN,60,60,2241,594,1203,188,496,247,273,59,245,331,66,45,128,165,1623,Brooklyn Nets,Dallas Mavericks,0,1,0,0,0,Brooklyn Nets,41.5,40.5,0.5060,113.80,113.30,0.445,"Brooklyn Nets, Dallas Mavericks",38.0,44.0,0.463,114.2,114.1,-0.14
3,Markieff Morris,10.1,1.3,3.3,0.409,0.8,2.0,0.394,0.5,1.3,0.432,0.530,0.3,0.3,1.000,0.4,1.7,2.0,0.9,0.3,0.1,0.5,1.4,3.8,10.334286,0.553486,0.613771,0.094114,4.011429,18.271429,11.234286,12.234286,1.240000,1.157143,13.191429,17.497143,0.077143,0.254286,0.308571,0.057543,-2.394286,-0.397143,-2.791429,-0.022857,33,NaN,35,2,355,47,115,28,71,10,10,13,58,31,9,5,18,48,132,Brooklyn Nets,Dallas Mavericks,0,0,1,0,0,Brooklyn Nets,41.5,40.5,0.5060,113.80,113.30,0.445,"Brooklyn Nets, Dallas Mavericks",38.0,44.0,0.463,114.2,114.1,-0.14
4,D'Angelo Russell,32.5,6.3,13.4,0.469,2.7,6.9,0.396,3.5,6.5,0.548,0.572,2.5,3.0,0.829,0.5,2.5,3.0,6.2,1.0,0.4,2.6,2.0,17.8,16.339437,0.605437,0.516451,0.228085,1.728169,8.500000,5.176056,27.430986,1.432394,1.071831,15.116901,22.752113,1.964789,1.236620,3.201408,0.106592,2.187324,-0.700000,1.487324,1.208451,26,NaN,71,71,2304,445,948,194,490,179,216,35,180,437,70,29,186,140,1263,Minnesota Timberwolves,Los Angeles Lakers,0,1,0,0,0,Minnesota Timberwolves,42.5,39.5,0.5180,116.50,116.20,0.105,"Minnesota Timberwolves, Los Angeles Lakers",43.0,39.0,0.524,117.2,116.6,0.43
...,...,

# Scraping the all_nba Category

In [1156]:
req = requests.get(f'https://www.basketball-reference.com/awards/all_league.html')
soup = BeautifulSoup(req.content, 'html.parser')
awards = []

# Extracting each player and their specific season
for row in soup.find('tbody').find_all('tr'):
    season = row.find('th', {'data-stat': 'season'})
    #need to make sure the season is there in order to get text and avoid errors
    if season:
        seasons = season.get_text()
    team = row.find('td', {'data-stat': 'all_team'})
    #need to make sure the team is there in order to get text and avoid errors
    if team:
        team_text = team.get_text()
        #was getting some blank values so needed to make sure there was something there otherwise it would be a null value that I can drop later
        if team_text:
            teams = team_text
     
    #the player were set up and counted from 1-15 so had to loop through it
    for i in range(1, 16):
        player = row.find('td', {'data-stat': str(i)})
        #same need to make sure the team is there in order to get text and avoid errors
        if player:
            player_found = player.find('a')
            if player_found:
                player_name = player_found.get_text()
                #need 
                player_dict = {
                    'season' : seasons,
                    'team' : teams,
                    'name' : player_name}
                #appending back to my list
                awards.append(player_dict)
#create a dataframe of list
df_all_nba = pd.DataFrame(awards)
df_all_nba.dropna(inplace = True)
df_all_nba = pd.get_dummies(df_all_nba, columns= ['team'], dtype = int)
df_all_nba

""


In [1144]:
def get_award(award_list):
    """
    Scrape basketball reference for player awards.

    This function takes a list of basketball awards and scrapes basketball-reference.com to retrieve the players who received those awards for each season.

    Parameters:
    -----------
    award_list : list
        A list of strings containing the names of awards.

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing information about the players who received the specified awards for each season. The DataFrame has columns for 'season' (season in which the award was received),
        'name' (name of the player receiving the award), and each award in `award_list`, where a value of 1 indicates that the player received the award for that season, and 0 indicates
        that they did not receive the award.

    Example:
    --------
    To scrape basketball-reference.com for MVP and Rookie of the Year awards:
    >>> awards_df = get_award(['mvp', 'roy'])
    """
    all_awards = pd.DataFrame()
    for award in award_list:
        req = requests.get(f'https://www.basketball-reference.com/awards/{award}.html')
        soup = BeautifulSoup(req.content, 'html.parser')
        awards = []
    
        # Extracting each player and their specific season
        for row in soup.find('tbody').find_all('tr'):
            award_dict = {}
            season = row.find('th', {'data-stat': 'season'})
            if season:
                award_dict['season'] = season.get_text()

            player = row.find('td', {'data-stat': 'player'})
            if player:
                player_found = player.find('a')
                if player_found:
                    award_dict['name'] = player_found.getText()
            #appending back to my list
            awards.append(award_dict)
        #create a dataframe of list
        df = pd.DataFrame(awards)
        df.reset_index(drop=True, inplace=True)
        #adding a binary column to match award
        df[award.upper()] = 1
        all_awards = pd.concat([all_awards, df], ignore_index=True)
        all_awards.fillna(0, inplace = True)
        df[award.upper()].astype(int)
    return all_awards

In [1145]:
#get awards dataframe
awards = ['mvp', 'dpoy', 'smoy', 'mip']
df = get_award(awards)

In [1146]:
#merge all_nba and awards dataframes
all_awards = pd.merge(df, df_all_nba, how = 'outer', on = ['season', 'name']).fillna(0)
all_awards

,season,name,MVP,DPOY,SMOY,MIP,team_1st,team_2nd,team_3rd
0,2023-24,Nikola Jokić,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2022-23,Joel Embiid,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2021-22,Nikola Jokić,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2020-21,Nikola Jokić,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2019-20,Giannis Antetokounmpo,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1132,1946-47,Frankie Baumholtz,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1133,1946-47,Ernie Calverley,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1134,1946-47,Chick Halbert,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1135,1946-47,John Logan,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Scrape Per Year

In [1147]:
def scrape_year(year, award_list):
    """
    Scrape and combine basketball player statistics and awards for a specific year.

    This function scrapes and combines player statistics and awards for a specific year, including total stats, advanced stats, and per game stats. It then merges these dataframes and saves the result to a CSV file.

    Parameters:
    -----------
    year : int
        The ending year of the season for which to retrieve the data to be scraped and combined.
    award_list : list of str
        A list of strings containing the names of awards.

    Returns:
    --------
    pandas.DataFrame
        The combined dataframe containing player statistics and awards for the specified year.

    Example:
    --------
    To scrape and combine player stats and the MVP and Rookie of the Year awards for the 2022-2023 season:
    >>> scrape_year(2023, ['mvp', 'roy'])
    """
    #scrape and clean total stats
    get_player_total_season_stats(year)
    df_total = pd.read_csv(f"./total_stats/{year-1}_{year}_player_season_totals.csv")
    df_total = combine_traded_player_total_stats(df_total)
    df_total = clean_total_stats(df_total)
    #scrape and clean advanced stats
    get_player_advanced_season_stats(year)
    df_advanced = pd.read_csv(f"./advanced_season_stat_total/{year-1}_{year}_advanced_player_season_totals.csv")
    df_advanced = combine_traded_player_advanced_stats(df_advanced)
    #scrape and clean per year stats
    get_per_game_stat(year)
    df_per = pd.read_csv(f'./per_game_stat/per_game_stat_{year-1}_{year}.csv')
    df_per = combine_traded_player_per_game_stats(df_per)
    
    #combine the total and advanced stats dataframes
    df = pd.merge(df_advanced, df_total, on = 'name')
    #combine the big dataframe and per year stats dataframes
    df = pd.merge(df_per, df, on = 'name')
    df = pd.get_dummies(df, columns=['positions'], dtype = int)
    #Combine team stats with big dataframe
    df_team = scrape_team_data(year)
    df_merge = pd.merge(df.dropna(subset=['team']), df_team, on='team', how = 'outer')
    df = combine_traded_player_team_data(df, df_team, df_merge)
    
    
    #adding season column for merge purposes with awards
    df['season'] = f'{year-1}-{str(year)[-2:]}'
    
    for award in award_list:
    #scrape awards
        accolade = get_award(award_list)
    all_awards = pd.merge(accolade, df_all_nba, how = 'outer', on = ['season', 'name']).fillna(0)
    #merge award and big df
    df = pd.merge(df, all_awards, how = 'left', on = ['season', 'name'])
    df.fillna(0, inplace = True)    
    
    df.to_csv(f'./total/total_{year-1}_{year}.csv', index = False)
    return df

In [ ]:
scrape_year(2022, ['mvp', 'dpoy', 'smoy', 'mip'])

HTTPError: 429 Client Error: Too Many Requests for url: https://www.basketball-reference.com/leagues/NBA_2022_totals.html

Not running the below cell again as I have already scraped the data needed.

In [ ]:
#scraping every season since 1979-1980 as that was the first year of the three point line. need to make the sleep time 60 because basketball reference has a strict per minute scrape policy
for i in range(1980, 2024):
    award_list = ['mvp', 'dpoy', 'smoy', 'mip']
    scrape_year(i, award_list)
    time.sleep(60)